<a href="https://colab.research.google.com/github/kdj0712/study_AIs/blob/main/docs/quests/MLs/SpineSurgeryList_FeatureEngin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 모델 추출 데이터 선정
- Target : 재발여부
- Features 선정
  + 범주형 : 혈액형, 성별
  + 수치형 : MF + ES, PI, PT, 체중, 신장

In [3]:
df_SSL = pd.read_csv("/content/drive/MyDrive/data_analytics/CSV파일/SpineSurgeryList.csv")
df_SSL[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [ ]:
df_SSL.isnull().sum()

In [5]:
print(df_SSL.value_counts())

Unnamed: 0  환자ID    Large Lymphocyte  Location of herniation  ODI   가족력  간질성폐질환  고혈압여부  과거수술횟수  당뇨여부  말초동맥질환여부  빈혈여부  성별  스테로이드치료  신부전여부  신장   심혈관질환  암발병여부  연령  우울증여부  입원기간  입원일자      종양진행여부  직업   체중    퇴원일자      헤모글로빈수치  혈전합병증여부  환자통증정도  흡연여부  통증기간(월)  수술기법  수술시간   수술실패여부  수술일자      재발여부  혈액형    전방디스크높이(mm)  후방디스크높이(mm)  지방축적도   Instability  MF + ES  Modic change  PI    PT    Seg Angle(raw)  Vaccum disc  골밀도    디스크단면적  디스크위치  척추이동척도  척추전방위증
0           1PT     22.8              3                       51.0  0.0  0       0      0       0     0         0     2   1        0      163  0      0      66  0      2     20190713  0       자영업  60.3  20190716  13.66    0        10      0     1.0      TELD  68.0   0       20190715  0     RH+A   16.1         12.3         282.3   0            1824.6   3             51.6  36.6  14.4            0            -1.01  2048.5  4      Down    0         1
576         577PT   32.1              1                       68.0  0.0  1       0      0       0     0

In [6]:
df_SSL.describe()

,Unnamed: 0,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,말초동맥질환여부,...,MF + ES,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추전방위증
count,1894.000000,1894.000000,1894.000000,462.000000,1843.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,...,1894.000000,1894.000000,1890.000000,1890.000000,1893.000000,1894.000000,998.000000,1893.000000,1894.000000,1894.000000
mean,946.500000,39.270750,2.295671,31.142857,0.049919,0.020063,0.130940,0.302534,0.062830,0.004224,...,2096.421600,0.247096,46.593651,16.329841,15.287934,0.056494,-1.507325,1989.788774,4.430834,0.032207
std,546.895023,13.675874,0.996143,10.131304,0.217836,0.140254,0.337424,0.638461,0.242721,0.064871,...,472.834488,0.661506,22.736211,8.421322,10.426397,0.230935,0.495867,314.478436,3.595680,0.176596
min,0.000000,0.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1016.800000,0.000000,11.900000,-10.600000,-27.400000,0.000000,-2.840000,1145.800000,1.000000,0.000000
25%,473.250000,30.700000,1.000000,24.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1765.285000,0.000000,38.700000,10.500000,8.800000,0.000000,-1.820000,1777.000000,4.000000,0.000000
50%,946.500000,40.200000,2.000000,31.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2056.500000,0.000000,45.100000,15.700000,14.500000,0.000000,-1.520000,1985.500000,4.000000,0.000000
75%,1419.750000,48.600000,3.000000,38.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2374.350000,0.000000,52.000000,21.400000,21.100000,0.000000,-1.202500,2178.500000,4.000000,0.000000
max,1893.000000,89.000000,5.000000,68.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,...,5569.000000,3.000000,559.000000,48.700000,165.000000,1.000000,1.700000,3214.200000,45.000000,1.000000


In [7]:
df_SSL['PT'].describe()

count    1890.000000
mean       16.329841
std         8.421322
min       -10.600000
25%        10.500000
50%        15.700000
75%        21.400000
max        48.700000
Name: PT, dtype: float64

In [8]:
# 'PI' 컬럼의 이상치 변환(다른 값들을 봤을 때 소수점을 누락했을 가능성이 높으므로 해당 값으로 치환)
df_SSL.loc[df_SSL['PI'] == 559, 'PI'] = 55.9

In [9]:
df_SSL['PI'].unique()

array([51.6, 40.8, 67.5, 49.2, 58.8, 41.8, 41.9, 50.6, 48.8, 38.9, 58.6,
       37.2, 42.9, 61.6, 49.9, 40.2, 38.8, 65.9, 32. , 37.1, 49. , 61. ,
       43.7, 41.4, 45.1, 52.9, 45.3, 51.2, 53.4, 36.1, 66.7, 55.3, 31.8,
       37.6, 54.2, 38.2, 35.8, 46. , 44. , 47. , 52.8, 49.7, 56.6, 36.3,
       44.1, 38.7, 61.5, 33.6, 84.6, 34.8, 50.3, 39.7, 59.6, 52. , 42.5,
       37.8, 46.7, 62. , 36.7, 39.5, 41.3, 31.3, 38.4, 34.6, 34.7, 44.6,
       41.7, 45.6, 27.7, 40.3, 32.2, 49.8, 53.8, 41.1, 40. , 33.3, 60.9,
       55.6, 51.3, 46.6, 33.7, 60.2, 47.3, 51.1, 38.5, 46.9, 77.6, 69.5,
       44.4, 43.6, 39.6, 37.3, 51.9, 45.9, 44.2, 39.8, 53. , 33.5, 29.5,
       54.8, 14. , 57.6, 49.5, 34.3, 58.2, 35.7, 29.7, 64.2, 50.5, 41.6,
       52.3, 48.1, 44.8, 32.1, 49.1, 46.5, 63.1, 49.4, 31.4, 35. , 30. ,
       53.9, 46.4, 36.8, 19.8, 62.6, 62.8, 67.4, 33.1, 56.7, 62.1, 62.4,
       56.8, 50.2, 36.5, 63.6, 24.9, 60.1, 36. , 58.7, 57.8, 55.5, 54.4,
       55.7, 40.5, 43.2, 43.5, 55.8, 34. , 43.1, 40

In [10]:
df_SSL.describe(include=object)

,환자ID,직업,수술기법,혈액형,척추이동척도
count,1894,1479,1813,1894,1894
unique,1894,17,2,4,5
top,1PT,사무직,TELD,RH+A,Middle
freq,1,449,1673,688,1260


In [11]:
df_SSL[['MF + ES','PI','PT','신장','체중','과거수술횟수','성별']].corr()

,MF + ES,PI,PT,신장,체중,과거수술횟수,성별
MF + ES,1.000000,-0.064920,-0.203665,0.190517,0.303566,0.003656,-0.248657
PI,-0.064920,1.000000,0.560199,-0.043853,-0.018940,0.081718,0.032053
PT,-0.203665,0.560199,1.000000,-0.048970,-0.025648,0.034926,0.076780
신장,0.190517,-0.043853,-0.048970,1.000000,0.703038,-0.001261,-0.739178
체중,0.303566,-0.018940,-0.025648,0.703038,1.000000,0.047395,-0.638322
과거수술횟수,0.003656,0.081718,0.034926,-0.001261,0.047395,1.000000,-0.040219
성별,-0.248657,0.032053,0.076780,-0.739178,-0.638322,-0.040219,1.000000


#### 데이터 추출

In [12]:
df_SSL_extract = df_SSL.loc[:,['MF + ES','PI','PT','신장','체중','재발여부','혈액형','성별']]
df_SSL_extract[:2]

,MF + ES,PI,PT,신장,체중,재발여부,혈액형,성별
0,1824.6,51.6,36.6,163,60.3,0,RH+A,2
1,1737.5,40.8,7.2,171,71.7,0,RH+A,1


In [13]:
df_SSL_extract.isnull().sum()

MF + ES    0
PI         4
PT         4
신장         0
체중         0
재발여부       0
혈액형        0
성별         0
dtype: int64

In [14]:
df_SSL_extract.describe()

,MF + ES,PI,PT,신장,체중,재발여부,성별
count,1894.000000,1890.000000,1890.000000,1894.000000,1894.000000,1894.000000,1894.000000
mean,2096.421600,45.795079,16.329841,168.027983,68.111140,0.119852,1.383316
std,472.834488,9.972554,8.421322,9.211419,11.234318,0.324874,0.486323
min,1016.800000,11.900000,-10.600000,140.000000,35.000000,0.000000,1.000000
25%,1765.285000,38.700000,10.500000,161.000000,59.700000,0.000000,1.000000
50%,2056.500000,45.100000,15.700000,169.000000,68.000000,0.000000,1.000000
75%,2374.350000,52.000000,21.400000,175.000000,75.000000,0.000000,2.000000
max,5569.000000,84.600000,48.700000,204.000000,136.100000,1.000000,2.000000


In [15]:
# 이상치는 없어 보임
df_SSL_extract['MF + ES'].unique()

array([1824.6 , 1737.5 , 1188.5 , 2534.5 , 1840.6 , 1683.6 , 1979.6 ,
       1789.6 , 1375.5 , 1601.9 , 1462.5 , 2636.91, 1967.7 , 1878.5 ,
       2012.5 , 1199.  , 1746.3 , 1930.7 , 2152.3 , 2101.3 , 1576.6 ,
       1724.5 , 2688.8 , 2183.9 , 2188.8 , 1458.3 , 1510.7 , 2097.4 ,
       1959.3 , 1514.2 , 2286.7 , 1633.6 , 1729.7 , 1669.7 , 2288.5 ,
       1897.1 , 1277.9 , 1747.2 , 2503.34, 2763.94, 3679.32, 1854.  ,
       2425.8 , 2501.4 , 1772.2 , 2677.2 , 2925.2 , 3755.4 , 1988.5 ,
       3057.2 , 2121.2 , 2718.2 , 2326.7 , 2054.2 , 2270.9 , 2481.2 ,
       2701.  , 3338.8 , 2104.1 , 2323.9 , 1583.1 , 2941.4 , 1801.1 ,
       2966.3 , 2082.  , 1851.2 , 2423.1 , 1704.  , 1867.5 , 1971.  ,
       1419.8 , 2016.6 , 1905.7 , 2747.5 , 2019.9 , 1978.6 , 1667.4 ,
       2295.5 , 1983.2 , 1649.3 , 2034.7 , 2545.5 , 1712.6 , 1815.8 ,
       2382.5 , 2463.2 , 2579.2 , 1690.8 , 2268.7 , 2734.3 , 2701.6 ,
       2910.9 , 1744.2 , 1602.3 , 1379.3 , 1798.84, 1382.96, 1585.77,
       1542.5 , 2098

In [16]:
df_SSL_extract['PT'].unique()

array([ 36.6,   7.2,  27.3,  18.7,  14.7,  23.6,  17.4,  18.5,  13.6,
        33.9,   8.9,  21.3,  19. ,  17.1,  14.1,  12.7,  17.5,  12.6,
        11.9,  31.4,  16.4,  20.8,  13.1,  16.1,  21.6,  25. ,  14.9,
        29. ,   9.9,  31. ,   6. ,  15.6,   3.8,  24. ,   9. ,  16. ,
        21.9,  27.6,   5.7,  11.6,   4.5,   7.5,  20.5,  29.2,  10.6,
        14.4,  24.6,   9.6,  22.5,  11.4,   4.2,  15.2,  12.9,  18.8,
         4.9,   2.2,   2. ,  25.6,  16.2,  13.9,  13.8,  10.1,   9.1,
        19.3,   6.4,  16.9,   9.8,  22.9,   8.6,   6.7,  25.7,  17.6,
        14.2,   8.1,  32.3,  29.3,  24.5,  22.3,   7.4,  16.3,  23.8,
         4.6,  19.6,  34.9,  36.7,  20.4,  17.8,   7.7,   1.5,  18.6,
         6.9,  22.8,  11.8,   3.6,   1.6,  15.3,  48.7,  27.2,   6.5,
        19.9,  13.2,  19.4,  10.9,  19.8,  21.4,  12.8,  25.1,   5.9,
        12.3,  15.7,  11.7,  40.8,  15.1,   4.4,  10.4,  28.2,  12.1,
        10.2,   2.9,  -4.8,  24.2,  30.8,  27. ,   4.7,  35.2,  22. ,
        15.5,   7.1,

In [17]:
df_SSL_extract['PI'].unique()

array([51.6, 40.8, 67.5, 49.2, 58.8, 41.8, 41.9, 50.6, 48.8, 38.9, 58.6,
       37.2, 42.9, 61.6, 49.9, 40.2, 38.8, 65.9, 32. , 37.1, 49. , 61. ,
       43.7, 41.4, 45.1, 52.9, 45.3, 51.2, 53.4, 36.1, 66.7, 55.3, 31.8,
       37.6, 54.2, 38.2, 35.8, 46. , 44. , 47. , 52.8, 49.7, 56.6, 36.3,
       44.1, 38.7, 61.5, 33.6, 84.6, 34.8, 50.3, 39.7, 59.6, 52. , 42.5,
       37.8, 46.7, 62. , 36.7, 39.5, 41.3, 31.3, 38.4, 34.6, 34.7, 44.6,
       41.7, 45.6, 27.7, 40.3, 32.2, 49.8, 53.8, 41.1, 40. , 33.3, 60.9,
       55.6, 51.3, 46.6, 33.7, 60.2, 47.3, 51.1, 38.5, 46.9, 77.6, 69.5,
       44.4, 43.6, 39.6, 37.3, 51.9, 45.9, 44.2, 39.8, 53. , 33.5, 29.5,
       54.8, 14. , 57.6, 49.5, 34.3, 58.2, 35.7, 29.7, 64.2, 50.5, 41.6,
       52.3, 48.1, 44.8, 32.1, 49.1, 46.5, 63.1, 49.4, 31.4, 35. , 30. ,
       53.9, 46.4, 36.8, 19.8, 62.6, 62.8, 67.4, 33.1, 56.7, 62.1, 62.4,
       56.8, 50.2, 36.5, 63.6, 24.9, 60.1, 36. , 58.7, 57.8, 55.5, 54.4,
       55.7, 40.5, 43.2, 43.5, 55.8, 34. , 43.1, 40

In [18]:
df_SSL_extract['신장'].unique()

array([163, 171, 178, 174, 183, 164, 175, 168, 160, 161, 162, 158, 176,
       188, 184, 166, 154, 179, 186, 173, 170, 156, 157, 167, 182, 151,
       169, 185, 153, 159, 180, 143, 155, 177, 146, 165, 144, 152, 181,
       172, 150, 189, 149, 147, 142, 187, 148, 145, 140, 190, 191, 204])

In [19]:
df_SSL_extract['체중'].unique()

array([ 60.3,  71.7,  77.1,  74.2,  80.7,  60.8,  74.8,  81.2,  69.2,
        63. ,  59.5,  64.9,  74.4,  74.6,  58.4,  75.6,  69.5,  85.2,
        82.1,  59.9,  68.3,  58.6,  57.2,  77.8,  69.8,  83.2,  82. ,
        78. ,  68.2,  73.3,  73.4,  70.9,  77.5,  71.9,  58.1,  58.2,
        61.6,  76.8,  57.9,  80.2,  56. ,  72.1,  73.6,  58.3,  59.3,
        71.3,  59.1,  75.2,  59.2,  70.1,  63.8,  69.6,  82.7,  69.4,
        72. ,  59.4,  58.5,  57. ,  71.5,  63.2,  75.7,  59.8,  62.6,
        59. ,  70.3,  75.3,  68.6,  78.6,  53.3,  57.5,  77. ,  77.9,
        57.1,  66.4,  71.8,  76.4,  78.1,  84.8,  60.2,  58.7,  54.5,
        81.4,  61.7,  57.3,  61.1,  78.2,  70.4,  64.6,  59.7,  56.8,
        60.6,  73.8,  66. ,  53.7,  61.4,  62.8,  67.8,  56.4,  67.2,
        79.6,  78.8,  64.2,  76. ,  61.5,  59.6,  56.7,  80. ,  81.8,
        81. ,  77.3,  73. ,  60.4,  74.7,  57.4,  60.7,  72.4,  73.7,
        79.9,  70.7,  74.3,  70.8,  57.6,  67.6,  81.6,  61.9,  55.8,
        63.9,  70.5,

In [20]:
df_SSL_extract['성별'].unique()

array([2, 1])

In [21]:
df_SSL_extract['재발여부'].unique()

array([0, 1])

In [22]:
df_SSL_extract.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MF + ES  1894 non-null   float64
 1   PI       1890 non-null   float64
 2   PT       1890 non-null   float64
 3   신장       1894 non-null   int64  
 4   체중       1894 non-null   float64
 5   재발여부     1894 non-null   int64  
 6   혈액형      1894 non-null   object 
 7   성별       1894 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 118.5+ KB


In [23]:
# 결측치 제거
df_SSL_extract = df_SSL_extract.dropna()
df_SSL_extract.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1890 entries, 0 to 1893
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   MF + ES  1890 non-null   float64
 1   PI       1890 non-null   float64
 2   PT       1890 non-null   float64
 3   신장       1890 non-null   int64  
 4   체중       1890 non-null   float64
 5   재발여부     1890 non-null   int64  
 6   혈액형      1890 non-null   object 
 7   성별       1890 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 132.9+ KB


### 수치형 Scaling(Min-Max Scaling으로 평준화)

In [24]:
from sklearn.preprocessing import MinMaxScaler
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(df_SSL_extract[['MF + ES','PI','PT','신장','체중']])

MinMaxScaler()

In [25]:
# 평준화한 내용으로 새로운 DataFrame 생성 : df_SSL_extract1
df_SSL_extract1 = pd.DataFrame(data=minMaxScaler.transform(df_SSL_extract[['MF + ES','PI','PT','신장','체중']]), columns=[['MF+ES_extract','PI_extract','PT_extract','신장_extract','체중_extract']])
df_SSL_extract1[:2]

,MF+ES_extract,PI_extract,PT_extract,신장_extract,체중_extract
0,0.177453,0.546080,0.795953,0.359375,0.250247
1,0.158319,0.397524,0.300169,0.484375,0.363007


In [26]:
# 내용확인
df_SSL_extract1.describe()

,MF+ES_extract,PI_extract,PT_extract,신장_extract,체중_extract
count,1890.000000,1890.000000,1890.000000,1890.000000,1890.000000
mean,0.236768,0.466232,0.454129,0.437599,0.327279
std,0.103619,0.137174,0.142012,0.143871,0.111077
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.163880,0.368638,0.355818,0.328125,0.244313
50%,0.228285,0.456671,0.443508,0.453125,0.325915
75%,0.297364,0.551582,0.539629,0.546875,0.395648
max,1.000000,1.000000,1.000000,1.000000,1.000000


### 범주형 Scaling(OneHot Encoding으로 평준화)

In [27]:
from sklearn.preprocessing import OneHotEncoder
oneHotEncoder_category = OneHotEncoder()
oneHotEncoder_category.fit(df_SSL_extract[['혈액형','성별']])

OneHotEncoder()

In [28]:
oneHotEncoder_category.categories_

[array(['RH+A', 'RH+AB', 'RH+B', 'RH+O'], dtype=object), array([1, 2])]

In [29]:
SSL_encoder = oneHotEncoder_category.transform(df_SSL_extract[['혈액형','성별']]).toarray()
SSL_encoder.shape

(1890, 6)

In [30]:
# 평준화된 내용으로 새로운 데이터프레임 생성 : df_SSL_encoder
df_SSL_encoder = pd.DataFrame(data=SSL_encoder, columns=oneHotEncoder_category.get_feature_names_out())
df_SSL_encoder[:2]

,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,성별_1,성별_2
0,1.0,0.0,0.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0


In [31]:
df_SSL_extract1.columns = ['MF+ES_extract', 'PI_extract', 'PT_extract', '신장_extract', '체중_extract']
df_SSL_extract2 = pd.concat([df_SSL_extract1.reset_index(drop=True), df_SSL_encoder.reset_index(drop=True)], axis=1)
df_SSL_extract2[:2]

,MF+ES_extract,PI_extract,PT_extract,신장_extract,체중_extract,혈액형_RH+A,혈액형_RH+AB,혈액형_RH+B,혈액형_RH+O,성별_1,성별_2
0,0.177453,0.546080,0.795953,0.359375,0.250247,1.0,0.0,0.0,0.0,0.0,1.0
1,0.158319,0.397524,0.300169,0.484375,0.363007,1.0,0.0,0.0,0.0,1.0,0.0


In [32]:
df_SSL_extract.shape

(1890, 8)

### 모델


#### 1. 수치형 Scaling 데이터로 먼저 산출

In [33]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model2 = LogisticRegression()

In [34]:
target_train1 = df_SSL_extract['재발여부']
features_train1 = df_SSL_extract1   # 수치형 자료만 담겨져 있는 데이터
target_train1.shape, features_train1.shape

((1890,), (1890, 5))

In [35]:
model1 = model1.fit(features_train1,target_train1)
model1

LogisticRegression()

In [36]:
model1.coef_, model1.intercept_
# 가중치  MF+ES_extract : 0.38914486, PI_extract : -0.68619067,	PT_extract : 0.26812755, 신장_extract : -0.62261001,	체중_extract : 3.7027376
# 절편 : -2.87912257

(array([[ 0.38914486, -0.68619067,  0.26812755, -0.62261001,  3.7027376 ]]),
 array([-2.87912257]))

#### 2. 수치형 + 범주형 Scaling 데이터로 모델 산출

In [37]:
target_train2 = df_SSL_extract['재발여부']
features_train2 = df_SSL_extract2   # 수치형 자료와 범주형 자료가 같이 담겨져 있는 데이터
target_train2.shape, features_train2.shape

((1890,), (1890, 11))

In [38]:
model2 = model2.fit(features_train2,target_train2)
model2

LogisticRegression()

In [39]:
# 대조군이 늘어나면서 가중치와 절편이 변경되었다.
model2.coef_, model2.intercept_
# 가중치 : (MF+ES_extract : 0.45775581), (PI_extract : -0.65813274), (PT_extract : 0.29622427), (신장_extract : -0.39295281), (체중_extract : 3.77792728 )
# 가중치 : (혈액형_RH+A : 0.01508906), (혈액형_RH+AB :  -0.03421861),	(혈액형_RH+B : 0.24803893), (혈액형_RH+O : -0.22810273), (성별_1 : -0.06362926), (성별_2 : 0.06443592)
# 절편 : -3.04297459

(array([[ 0.45775581, -0.65813274,  0.29622427, -0.39295281,  3.77792728,
          0.01508906, -0.03421861,  0.24803893, -0.22810273, -0.06362926,
          0.06443592]]),
 array([-3.04297459]))

### 평가

### 1. Model 1으로 수치 산출 및 평가

In [44]:
df_SSL_extract['재발여부'][10:20]

10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: 재발여부, dtype: int64

In [45]:
model1.predict(features_train1[10:20])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [46]:
model1.predict_proba(features_train1[10:20])

array([[0.91587624, 0.08412376],
       [0.88186437, 0.11813563],
       [0.86223187, 0.13776813],
       [0.88400168, 0.11599832],
       [0.91257772, 0.08742228],
       [0.86786847, 0.13213153],
       [0.87805677, 0.12194323],
       [0.85960793, 0.14039207],
       [0.8277696 , 0.1722304 ],
       [0.90243744, 0.09756256]])

In [48]:
# 정확도
from sklearn.metrics import accuracy_score
target_train_predict1 = model1.predict(features_train1)
target_train_predict1.shape

(1890,)

In [50]:
# 정상데이터와, 모델링 적용 데이터를 대입시켜 정확도 산출
accuracy_score1 = accuracy_score(target_train1, target_train_predict1)
accuracy_score1

0.8804232804232804

In [51]:
# F1 Score 적용
from sklearn.metrics import classification_report
print(classification_report(target_train1, target_train_predict1))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1663
           1       1.00      0.00      0.01       227

    accuracy                           0.88      1890
   macro avg       0.94      0.50      0.47      1890
weighted avg       0.89      0.88      0.82      1890



### 2. Model 2로 수치 산출 및 평가

In [52]:
df_SSL_extract['재발여부'][10:20]

10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
Name: 재발여부, dtype: int64

In [53]:
model2.predict(features_train2[10:20])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [54]:
model2.predict_proba(features_train2[10:20])

array([[0.89086105, 0.10913895],
       [0.91061646, 0.08938354],
       [0.86455227, 0.13544773],
       [0.89005908, 0.10994092],
       [0.92745589, 0.07254411],
       [0.84249843, 0.15750157],
       [0.8837055 , 0.1162945 ],
       [0.85407461, 0.14592539],
       [0.78927514, 0.21072486],
       [0.91818432, 0.08181568]])

In [55]:
# 정확도
target_train_predict2 = model2.predict(features_train2)
target_train_predict2.shape

(1890,)

In [56]:
# 정상데이터와, 모델링 적용 데이터를 대입시켜 정확도 산출
accuracy_score2 = accuracy_score(target_train2, target_train_predict2)
accuracy_score2

0.8804232804232804

In [57]:
# F1 Score 적용
print(classification_report(target_train2, target_train_predict2))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1663
           1       1.00      0.00      0.01       227

    accuracy                           0.88      1890
   macro avg       0.94      0.50      0.47      1890
weighted avg       0.89      0.88      0.82      1890



In [58]:
# 최종 비교
print(classification_report(target_train1, target_train_predict1)), print(classification_report(target_train2, target_train_predict2))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1663
           1       1.00      0.00      0.01       227

    accuracy                           0.88      1890
   macro avg       0.94      0.50      0.47      1890
weighted avg       0.89      0.88      0.82      1890

              precision    recall  f1-score   support

           0       0.88      1.00      0.94      1663
           1       1.00      0.00      0.01       227

    accuracy                           0.88      1890
   macro avg       0.94      0.50      0.47      1890
weighted avg       0.89      0.88      0.82      1890



(None, None)

## 총평
- model1의 F1 score와 model2의 F1 score의 값이 정확히 일치하고, 그 오차가 매우 크며, 변경된 점이 없음을 통해 크게 두가지 정도의 상황을 유추할 수 있다.
  + 1. model 도출을 위한 데이터 선정 과정에서 target에 영향을 주지 않는 대조군을 선정하였을 가능성
  + 2. 또는, 데이터 자체가 영향을 주지 못할 정도로 target이 편향성을 가진 값을 가진 구조인 경우
- 이에 따라, 다른 범주형 데이터(category)모델에 적용하여 유의미한 값이 나오는지 확인하고 비교하는 방법을 제안할 수 있다.